# DiploDatos 2021


### Categorización de publicaciones de productos realizadas en Mercado Libre

### 02 - Análisis y Curación

#### Condiciones generales que aplican a todos los prácticos:
   - Las notebooks tienen que ser 100% reproducibles, es decir al ejecutar las celdas tal cuál como se entrega la notebook se deben obtener los mismos resultados sin errores.
   - Código legible, haciendo buen uso de las celdas de la notebook y en lo posible seguir estándares de código para *Python* (https://www.python.org/dev/peps/pep-0008/).
   - Utilizar celdas tipo *Markdown* para ir guiando el análisis.
   - Limpiar el output de las celdas antes de entregar el notebook (ir a *Kernel* --> *Restart Kernel and Clear All Ouputs*).
   - Incluir conclusiones del análisis que se hizo en la sección "Conclusiones". Tratar de aportar valor en esta sección, ser creativo.

## 1. Consignas

#### Sección A:  Limpieza de texto / Preprocessing

Tener en cuenta lo siguiente: 

1. *Unidecode*

2. Pasar a minúsculas

3. Limpiar números

4. Limpiar símbolos **(** ' ! ¡ " @ % & * , . : ; < = > ¿ ? @ \ ^ _ { | } ~ \t \n [ ] ` $ **)**

5. Limpiar caracteres que suelen usarse como espacios **(** ' + ( ) - \ **)**

6. Reemplazar contracciones, por ejemplo, **c/u** por *cada uno*, **c/** por *con*, **p/** por *para*.

7. Etc.

#### Sección B: Tokenización & Secuencias

1. Utilizar métodos `fit_on_texts()`, `texts_to_sequences()`, y `pad_sequences()`:

- https://keras.io/api/preprocessing/text/#tokenizer-class

- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

#### Sección C: Label Encoding

1. Utilizar método `LabelEncoder()` de *sklearn*:

- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

#### Sección D: Word Embeddings

Generar los *word embeddings* correspondientes, de las siguientes dos formas:

1. *Custom Word Embeddings*
2. *Loading Pretrained Word Embeddings* (**opcional**)

En ambos puntos el objetivos final es llegar a crear la *embedding layer* de *keras*:

- https://keras.io/api/layers/core_layers/embedding/

## 2. Código y Análisis

Importaciones necesarias.

In [ ]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

Lectura de dataset reducido.

In [ ]:
df_dataset = pd.read_csv('DataSet/dataset.csv')

Estudiamos el dataset brevemente antes de comenzar a operar sobre el mismo.

In [ ]:
df_dataset.describe()

In [ ]:
classes = np.sort(df_dataset.category.unique())

print(f'Dimensiones: {df_dataset.shape}')
print('----------')
print(f'Variables: {list(df_dataset.columns)}')
print('----------')
print(f'Categorías: {list(classes)}')

## Sección A

Antes de aplicar la limpieza, demos un vistazo a algunas de las publicaciones de nuestro conjunto de datos.

In [ ]:
df_dataset.sample(10, random_state=123)

**Aplicamos Limpieza**

Se define la serie de operaciones para la limpieza de títulos de publicaciones.

In [ ]:
def cleaning(title):
    """
    Aplica las operaciones de limpieza a un título de una publicación.
    """
    # Unidecode: Convierte string de Unicode a ASCII.
    title = unidecode(title)
    # Pasamos a Minúsculas.
    title = title.lower()
    # Eliminamos Números.
    title = re.sub(r'[0-9]+', '', title)
    # Reemplazamos Contracciones.
    title = re.sub(r'c/u', 'cada uno', title)
    title = re.sub(r'c/', 'con', title)
    title = re.sub(r'p/', 'para', title)
    # Limpiamos Símbolos.
    title = re.sub('[^a-zA-Z ]', '', title)
    # Retornamos el título de la publicación procesado.
    return title

In [ ]:
df_dataset['clean_title'] = df_dataset.title.apply(cleaning)

**Limpieza Definitiva**

Damos un vistazo al resultado del procesamiento, luego de haber aplicado todos los pasos anteriores.

In [ ]:
df_dataset.sample(10, random_state=123)

**Observación sobre Unidecode**

A simple vista, se eliminan los tildes (en ambos idiomas).

Desde la [documentación](https://pypi.org/project/Unidecode/), se especifica:

It often happens that you have text data in *Unicode*, but you need to represent it in *ASCII*.

What **Unidecode** provides is a middle road: the function `unidecode()` takes *Unicode* data and tries to represent it in *ASCII* characters (i.e., the universally displayable characters between `0x00` and `0x7F`), where the compromises taken when mapping between two character sets are chosen to be near what a human with a *US* keyboard would choose.

## Sección B

Separamos nuestro conjunto de datos en los vectores `X`, e `y`.

- El primero, `X`, comprende los títulos procesados de las publicaciones.

- El segundo, `y`, representa las categorías de las publicaciones.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

X = df_dataset.clean_title.values
y = df_dataset.category.values

X[0], y[0]

Utilizamos `Tokenizer()` para convertir los títulos de publicaciones en vectores.

In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X)

Necesitamos conocer el tamaño de nuestro vocabulario (se suma `+ 1` para contemplar las palabras *out of vocabulary*).

In [ ]:
vocab_length = len(word_tokenizer.word_index) + 1

vocab_length

Cada palabra se transforma al correspondiente índice en nuestro vocabulario.

In [ ]:
embedded_sentences = word_tokenizer.texts_to_sequences(X)

embedded_sentences[0]

Aplicamos *padding* para que todos los vectores de palabras tengan tamaños equivalentes.

In [ ]:
padded_sentences = pad_sequences(embedded_sentences, padding='post')

padded_sentences[0]

In [ ]:
ammount_sentences, sentences_length = padded_sentences.shape

ammount_sentences, sentences_length

## Sección C

Necesitamos codificar las categorías de nuestras publicaciones.
Por lo tanto, utilizamos `LabelEncoder()` para transformar los nombres en valores numéricos.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(classes)

¿Cuántas categorías identificó el *encoder*?

In [ ]:
le.classes_

In [ ]:
le.transform(classes)

Aplicamos la transformación aprendida a todas las categorías de nuestro conjunto de datos.

In [ ]:
encoded_labels = le.transform(y)

## Sección D

Finalmente, solo restan obtener los *word embeddings* para los títulos de nuestras publicaciones.
Utilizaremos `Embedding()` para calcularlos.

**Custom Word Embeddings**

De manera arbitraria, los vectores resultantes serán de **25** dimensiones.

In [ ]:
from keras.layers.embeddings import Embedding

embedding_layer = Embedding(vocab_length, 25, input_length=sentences_length)

**Pretrained Word Embeddings**

Desde https://nlp.stanford.edu/projects/glove/, se descarga el *word embeding* entrenado **glove.6B.zip**.

De manera arbitraria, utilizaremos los vectores de **100** dimensiones.

In [ ]:
embedding_dim = 100
glove_path = f'DataSet/glove.6B/glove.6B.{embedding_dim}d.txt'
glove_file = open(glove_path, encoding='utf8')

# Prepare embedding dictionary
embeddings_dictionary = dict()
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions

glove_file.close()

In [ ]:
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_length, embedding_dim))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        embedding_matrix[index] = embedding_vector
        hits += 1
    else:
        misses += 1

print(f'Converted {hits} words ({misses} misses)')

In [ ]:
trained_embedding_layer = Embedding(vocab_length,
                                    embedding_dim,
                                    weights=[embedding_matrix],
                                    input_length=sentences_length,
                                    trainable=False)

## 3. Conclusiones

En el laboratorio, nos concentramos principalmente en la curación de títulos de publicaciones en nuestro conjunto de datos, preparando la información para el aprendizaje de un futuro modelo.
Por lo tanto, no contamos con demasiadas conclusiones sobre el procesamiento realizado.
De todas formas, a continuación listaremos algunas observaciones interesantes.

- La limpieza de títulos resulta una tarea compleja, y podría ser necesario regresar a esta etapa para refinarla.
- El tamaño de nuestro vocabulario es **97180**. Hay que tener en cuenta que estamos trabajando con dos idiomas al mismo tiempo. Quizás podría ser necesario reducir su tamaño, limitándonos a las palabras más comunes.
- Algunos parámetros fueron definidos de manera totalmente arbitraria, como las dimensiones de nuestro *word embedding* (**25**), o las dimensiones de los *word vectors* preentrenados (**100**).
- Se utilizan los [GloVe](https://nlp.stanford.edu/projects/glove/) como nuestros *word vectors* preentrenados. Hay que notar que los vectores están preparados para trabajar con documentos en inglés (razón por la cual convertimos solo **28573** palabras, y perdemos **68606**). Una alternativa, sería utilizar [fastText](https://fasttext.cc/docs/en/pretrained-vectors.html).

### Aprendizaje Automático...

Para practicar un poco con nuestra implementación, intentaremos predecir con lo que tenemos hasta este punto.

**Definición del Modelo**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model = Sequential()

model.add(trained_embedding_layer)
model.add(Flatten())
model.add(Dense(len(classes), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

**Preprocesamiento**

Necesitaríamos un paso adicional de procesamiento para utilizar nuestras categorías en el modelo definido.

In [ ]:
from keras.utils import to_categorical

one_hot_labels = to_categorical(encoded_labels, num_classes=len(classes))

encoded_labels[0], one_hot_labels[0]

**Entrenamiento del Modelo**

In [ ]:
# Ya que este proceso puede demorar, realizaremos un entrenamiento breve
model.fit(padded_sentences, one_hot_labels, epochs=1, verbose=1)

In [ ]:
# Evaluamos contra nuestro propio conjunto de entrenamiento
loss, accuracy = model.evaluate(padded_sentences, one_hot_labels, verbose=0)

print(f'Accuracy: {accuracy * 100}')

**Predicción**

Se define un conjunto de datos de test improvisado, para analizar si el modelo aprende como nosotros esperamos.

In [ ]:
X_test = np.array(
    [
        'silla bebe auto',
        'maquina cafe taza',
        'cama dormir colchon',
        'musica teclado teclas',
        'jean pantalon talle',
        'perro golden macho',
        'heladera freezer frio',
        'vino estancia uva'
    ]
)

y_test = np.array(
    [
        'BABY_CAR_SEATS',
        'COFFEE_MAKERS',
        'MATTRESSES',
        'MUSICAL_KEYBOARDS',
        'PANTS',
        'PUREBRED_DOGS',
        'REFRIGERATORS',
        'WINES'
    ]
)

Debemos aplicar el mismo procesamiento del conjunto de entrenamiento, al conjunto improvisado de test (los títulos de las publicaciones no necesitan ser curados, ya que fueron definidos de forma tal para evitar este paso).

In [ ]:
embedded_sentences_test = word_tokenizer.texts_to_sequences(X_test)
padded_sentences_test = pad_sequences(embedded_sentences_test, sentences_length, padding='post')

embedded_sentences_test[0], padded_sentences_test[0]

In [ ]:
encoded_labels_test = le.transform(y_test)
one_hot_labels_test = to_categorical(encoded_labels_test, num_classes=len(classes))

encoded_labels_test[0], one_hot_labels_test[0]

In [ ]:
# Intentamos predecir los datos improvisados
predictions = model.predict(padded_sentences_test, verbose=0)

print(f'Predicción - {list(np.argmax(predictions, axis=-1))}')
print(f'Categorías - {list(encoded_labels_test)}')

Salvo las publicaciones de `MATTRESSES` y `MUSICAL_KEYBOARDS`, nuestro modelo predijo correctamente los datos de juguete.

#### Material de ayuda para el desarrollo de este práctico:

1. Implementación en *keras* de *word embeddings*: https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras
2. Como utilizar *pre-trained word embeddings* en *keras*: https://keras.io/examples/nlp/pretrained_word_embeddings/
3. *Word Embeddings*: https://jalammar.github.io/illustrated-word2vec/
3. Curso de **procesamiento del lenguaje natural** con *keras*: https://www.coursera.org/learn/natural-language-processing-tensorflow/home/welcome